In [0]:
%pip install git+https://github.com/databrickslabs/dbignite.git

In [0]:
# %pip install git+https://github.com/databricks-industry-solutions/redox-ehr-api

# Databricks Health & Life Sciences
### Webinar FY25 Q4
## Prior Authorization Review with DBSQL, Variant, dbignite & redox-ehr-api

*** 

## Write Back Claim Response
***

In [0]:
%sql
USE redox.hls_webinar_fy25q4;

In [0]:
df = spark.table("prior_auth_request_bundle")
display(df)

In [0]:
resources = ["Claim", "Patient", "Practitioner", "Organization", "Coverage", "RelatedPerson", "ServiceRequest"]
resources

In [0]:
from pyspark.sql.functions import schema_of_variant, schema_of_variant_agg, variant_get
from pyspark.sql.functions import col, regexp_replace

schemas = (
  df
  .groupBy(col("bundle_uuid"))
  .agg(*[regexp_replace(schema_of_variant_agg(col(r)), 'OBJECT', 'STRUCT')
    .alias(f"{r}_schema") for r in resources])
)

schema_dict = {f"{r}": row[f"{r}_schema"] for row in schemas.collect() for r in resources}

# example for Claim
schema_dict.get("Claim")

In [0]:
from pyspark.sql.functions import cast, to_json, from_json

df_transformed = (
  df.select(
    col("bundle_uuid")
    ,*[from_json(to_json(col(r)), schema_dict.get(r)).alias(r) for r in resources]
  )
)

display(df_transformed)

In [0]:
from pyspark.sql.functions import lit, struct, array, expr, col, concat

claimResponse = (
  df_transformed
  .withColumn("id", expr("uuid()"))
  .withColumn("claimResponse_id", concat(lit("/databricks/mgiglia/"), col("id")))
  .withColumn("claimResponse_status", lit("active"))
  .withColumn("claimResponse_use", lit("preauthorization"))
  .withColumn("claimResponse_type_code", col("Claim")[0].resource.type.coding[0].code)
  .withColumn("claimResponse_type_system", lit("http://terminology.hl7.org/CodeSystem/claim-type"))
  .withColumn("adjudication_system", lit("http://terminology.hl7.org/CodeSystem/adjudication"))
  .withColumn("adjudication_code", lit("approved"))
  .withColumn("claimResponse_text", lit("Requested preauthorization has been approved."))
)  

display(claimResponse)

In [0]:
from dbignite.writer.bundler import *
from dbignite.writer.fhir_encoder import *

In [0]:
maps = [
  Mapping('claimResponse_id', 'ClaimResponse.id')
  ,Mapping('claimResponse_status', 'ClaimResponse.status')
  ,Mapping('claimResponse_use', 'ClaimResponse.use')
  ,Mapping('claimResponse_type_code', 'ClaimResponse.type.coding.code')
  ,Mapping('claimResponse_type_system', 'ClaimResponse.type.coding.system')
  ,Mapping('adjudication_system', 'ClaimResponse.item.adjudication.category.coding.system')
  ,Mapping('adjudication_code', 'ClaimResponse.item.adjudication.category.coding.code')
  ,Mapping('claimResponse_text', 'ClaimResponse.text')
]

In [0]:
# import json
# from pyspark.sql.functions import udf, collect_list, struct, to_json
# from pyspark.sql.types import StringType, StructType, StructField, ArrayType

# class BundleServerless():
#     def __init__(self, mm):
#         self.mm = mm

#     def df_to_fhir(self, df):
#         return self._encode_to_json(self._encode_df(df))

#     def _encode_df(self, df):
#         encode_udf = udf(lambda row: [self.mm.encode(row, resourceType) for resourceType in self.mm.fhir_resource_list()], 
#                          ArrayType(StructType([StructField("resource", StringType())])))
        
#         return df.withColumn("encoded", encode_udf(struct(*df.columns)))

#     def _encode_to_json(self, df):
#         resource_to_fhir_udf = udf(self._resource_to_fhir, StringType())
        
#         return (df
#             .select(collect_list(resource_to_fhir_udf("encoded")).alias("entry"))
#             .withColumn("resourceType", lit("Bundle"))
#             .select(to_json(struct("resourceType", "entry")).alias("bundle"))
#         )

#     def _resource_to_fhir(self, resource):
#         resource_dict = json.loads(resource)
#         resource_type = list(resource_dict.keys())[0]
#         return json.dumps({
#             'resource': {
#                 'resourceType': resource_type,
#                 **resource_dict[resource_type]
#             }
#         })


In [0]:
# Instance of the encoder & bundle writer
#  - Encoder transforms data to valid FHIR format in Spark
#  - bundler maps data to json format
m = MappingManager(maps, claimResponse.schema)
b = Bundle(m)
result = b.df_to_fhir(claimResponse)

In [0]:
print('\n'.join([str(x) for x in 
       result.map(lambda x: json.loads(x)).map(lambda x: json.dumps(x, indent=4)).take(10)]))

In [0]:
import json

# Convert the DataFrame to JSON strings
result_json = result.selectExpr("to_json(struct(*)) as json_string")

# Collect the JSON strings
json_strings = result_json.collect()

# Pretty print the JSON strings
for row in json_strings:
    json_string = row.json_string
    parsed_json = json.loads(json_string)
    print(json.dumps(parsed_json, indent=4))

In [0]:
display(result)

In [0]:
# Create a dummy Dataframe with 2 rows of data
data = spark.createDataFrame([('CLM123', 'PAT01', 'COH123'), ('CLM345', 'PAT02', 'COH123')],['CLAIM_ID', 'PATIENT_ID', 'PATIENT_COHORT_NUM'])

# Define a mapping from DF columns to FHIR Schema, including a hardcoded value for Patient.identifier.system
maps = [Mapping('CLAIM_ID', 'Claim.id'), 
		Mapping('PATIENT_COHORT_NUM', 'Patient.identifier.value'),
    Mapping('<url of a hardcoded system reference>', 'Patient.identifier.system', True),
		Mapping('PATIENT_ID', 'Patient.id')]

# Instance of the encoder & bundle writer
#  - Encoder transforms data to valid FHIR format in Spark
#  - bundler maps data to json format
m = MappingManager(maps, data.schema)
b = Bundle(m)
result = b.df_to_fhir(data)

#Pretty printing the resulting RDD
import json
result.map(lambda x: json.loads(x)).foreach(lambda x: print(json.dumps(x, indent=4)))

In [0]:
   
    ["resourceType": "ClaimResponse",
        "id": "RedoxPAClaimResponseExample",
        "status": "active",
        "use": "preauthorization",
        "item": [
          {
            "adjudication": [
              {
                "category": {
                  "coding": [
                    {
                      "system": "http://terminology.hl7.org/CodeSystem/adjudication",
                      "code": "submitted"
                    }
                  ]
                },
                "extension": [
                  {
                    "extension": [
                      {
                        "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-reviewActionCode",
                        "valueCodeableConcept": {
                          "coding": [
                            {
                              "code": "A1",
                              "system": "https://codesystem.x12.org/005010/306",
                              "display": "Certified in total"
                            }
                          ]
                        }
                      },
                      {
                        "url": "number",
                        "valueString": "AUTH1001"
                      }
                    ],
                    "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-reviewAction"
                  }
                ]
              }
            ],
            "extension": [
              {
                "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-administrationReferenceNumber",
                "valueString": "REF456"
              },
              {
                "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemPreAuthIssueDate",
                "valueDate": "2021-08-23"
              },
              {
                "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemPreAuthPeriod",
                "valuePeriod": {
                  "start": "2021-08-23",
                  "end": "2021-09-23"
                }
              },
              {
                "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemTraceNumber",
                "valueIdentifier": {
                  "value": "23462346"
                }
              },
              {
                "extension": [
                  {
                    "url": "productOrServiceCode",
                    "valueCodeableConcept": {
                      "coding": [
                        {
                          "code": "93451",
                          "system": "http://www.ama-assn.org/go/cpt"
                        }
                      ],
                      "text": "Right heart catheterization"
                    }
                  },
                  {
                    "url": "quantity",
                    "valueQuantity": {
                      "value": 1
                    }
                  },
                  {
                    "url": "unitPrice",
                    "valueMoney": {
                      "value": 4966,
                      "currency": "USD"
                    }
                  }
                ],
                "url": "http://hl7.org/fhir/us/davinci-pas/StructureDefinition/extension-itemAuthorizedDetail"
              }
            ],
            "itemSequence": 1
          }
        ],
        "type": {
          "coding": [
            {
              "code": "professional",
              "system": "http://terminology.hl7.org/CodeSystem/claim-type"
            }
          ]
        },
        "patient": {
          "reference": "Patient/RedoxPABeneficiaryExample"
        },
        "created": "2021-08-23T16:35:54.648Z",
        "insurer": {
          "reference": "Organization/RedoxPAInsurerOrganizationExample"
        },
        "requestor": {
          "reference": "Organization/RedoxPARequestorOrganizationExample"
        },
        "request": {
          "reference": "Claim/RedoxPAServiceClaimExample",
          "identifier": {
            "type": {
              "coding": [
                {
                  "code": "ClaimID"
                }
              ]
            },
            "value": "urn:uuid:97b8d6d0-ac3d-411a-b32a-e1d237aceb6a"
          }
        },
        "outcome": "complete",
        "processNote": [
          {
            "text": "Covered under extended benefits"
          }
        ]
      }
    }